In [20]:
import torch.nn as nn
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler



n_in = 5
n_out = 1
n_hidden = 8

df = pd.read_csv('assets/Housing.csv')

X = df[['area', 'bedrooms', 'bathrooms', 'stories', 'parking']].values
y = df['price'].values

model = nn.Sequential(
    nn.Linear(n_in, n_hidden),
    nn.ReLU(),
    nn.Linear(n_hidden, n_out)
)

print(model)

Sequential(
  (0): Linear(in_features=5, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)


In [13]:
#use gpu if available  
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

X_tensor = X_tensor.to(device)
y_tensor = y_tensor.to(device)
print(X_tensor.shape, y_tensor.shape)

model = model.to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

#training loop
n_epochs = 100
for epoch in range(n_epochs):
    #forward pass
    y_pred = model(X_tensor)
    loss = criterion(y_pred, y_tensor)

    #backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')





Using device: cuda
torch.Size([545, 5]) torch.Size([545])
Epoch 1, Loss: 24693662810112.0
Epoch 11, Loss: 24572778774528.0
Epoch 21, Loss: 24446853185536.0
Epoch 31, Loss: 24315298840576.0
Epoch 41, Loss: 24177698406400.0
Epoch 51, Loss: 24033787641856.0
Epoch 61, Loss: 23883411357696.0
Epoch 71, Loss: 23726489862144.0
Epoch 81, Loss: 23562989600768.0
Epoch 91, Loss: 23392910573568.0


/home/dman/master/lib/python3.12/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([545])) that is different to the input size (torch.Size([545, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [16]:
import torch.nn as nn
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Model parameters
n_in = 5
n_out = 1
n_hidden = 8

# Load and preprocess data
df = pd.read_csv('assets/Housing.csv')

X = df[['area', 'bedrooms', 'bathrooms', 'stories', 'parking']].values
y = df['price'].values

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled)
y_train_tensor = torch.FloatTensor(y_train).reshape(-1, 1)
X_val_tensor = torch.FloatTensor(X_val_scaled)
y_val_tensor = torch.FloatTensor(y_val).reshape(-1, 1)

# Define the model
model = nn.Sequential(
    nn.Linear(n_in, n_hidden),
    nn.ReLU(),
    nn.Linear(n_hidden, n_out)
)

# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training parameters
n_epochs = 100
batch_size = 32

# Training loop with validation
for epoch in range(n_epochs):
    # Training
    model.train()
    train_loss = 0
    n_batches = len(X_train_tensor) // batch_size
    
    for i in range(n_batches):
        start_idx = i * batch_size
        end_idx = start_idx + batch_size
        
        batch_X = X_train_tensor[start_idx:end_idx]
        batch_y = y_train_tensor[start_idx:end_idx]
        
        # Forward pass
        y_pred = model(batch_X)
        loss = criterion(y_pred, batch_y)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    
    train_loss /= n_batches
    
    # Validation
    model.eval()
    with torch.no_grad():
        val_pred = model(X_val_tensor)
        val_loss = criterion(val_pred, y_val_tensor)
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{n_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

# Final evaluation
model.eval()
with torch.no_grad():
    train_pred = model(X_train_tensor)
    val_pred = model(X_val_tensor)
    
    final_train_loss = criterion(train_pred, y_train_tensor)
    final_val_loss = criterion(val_pred, y_val_tensor)

print(f'\nFinal Results:')
print(f'Training Loss: {final_train_loss:.4f}')
print(f'Validation Loss: {final_val_loss:.4f}')

Epoch [10/100], Train Loss: 25208856591123.6914, Val Loss: 30129984110592.0000
Epoch [20/100], Train Loss: 25208845782724.9219, Val Loss: 30129971527680.0000
Epoch [30/100], Train Loss: 25208828198912.0000, Val Loss: 30129952653312.0000
Epoch [40/100], Train Loss: 25208803194407.3828, Val Loss: 30129919098880.0000
Epoch [50/100], Train Loss: 25208768833378.4609, Val Loss: 30129883447296.0000
Epoch [60/100], Train Loss: 25208722857353.8477, Val Loss: 30129822629888.0000
Epoch [70/100], Train Loss: 25208665911611.0781, Val Loss: 30129757618176.0000
Epoch [80/100], Train Loss: 25208598964066.4609, Val Loss: 30129673732096.0000
Epoch [90/100], Train Loss: 25208522659997.5391, Val Loss: 30129579360256.0000
Epoch [100/100], Train Loss: 25208438128640.0000, Val Loss: 30129476599808.0000

Final Results:
Training Loss: 25234363121664.0000
Validation Loss: 30129476599808.0000


In [22]:
import torch.nn as nn
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Model parameters
n_in = 5
n_out = 1
n_hidden = 8

# Load and preprocess data
df = pd.read_csv('assets/Housing.csv')

X = df[['area', 'bedrooms', 'bathrooms', 'stories', 'parking']].values
y = df['price'].values

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
X_scaler = StandardScaler()
X_train_scaled = X_scaler.fit_transform(X_train)
X_val_scaled = X_scaler.transform(X_val)

# Scale the target values
y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train.reshape(-1, 1))
y_val_scaled = y_scaler.transform(y_val.reshape(-1, 1))

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled)
y_train_tensor = torch.FloatTensor(y_train_scaled)
X_val_tensor = torch.FloatTensor(X_val_scaled)
y_val_tensor = torch.FloatTensor(y_val_scaled)

#use gpu if available  
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

X_train_tensor = X_train_tensor.to(device)
y_train_tensor = y_train_tensor.to(device)
X_val_tensor = X_val_tensor.to(device)
y_val_tensor = y_val_tensor.to(device)



# Define the model
model = nn.Sequential(
    nn.Linear(n_in, n_hidden),
    nn.ReLU(),
    nn.Linear(n_hidden, n_out)
)
model = model.to(device)
# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training parameters
n_epochs = 100
batch_size = 32

# Lists to store losses for plotting
train_losses = []
val_losses = []

# Training loop with validation
for epoch in range(n_epochs):
    # Training
    model.train()
    train_loss = 0
    n_batches = len(X_train_tensor) // batch_size
    
    for i in range(n_batches):
        start_idx = i * batch_size
        end_idx = start_idx + batch_size
        
        batch_X = X_train_tensor[start_idx:end_idx]
        batch_y = y_train_tensor[start_idx:end_idx]
        
        # Forward pass
        y_pred = model(batch_X)
        loss = criterion(y_pred, batch_y)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    
    train_loss /= n_batches
    train_losses.append(train_loss)
    
    # Validation
    model.eval()
    with torch.no_grad():
        val_pred = model(X_val_tensor)
        val_loss = criterion(val_pred, y_val_tensor)
        val_losses.append(val_loss.item())
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{n_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

#Final evaluation with actual price values
model.eval()
with torch.no_grad():
    # Get predictions
    train_pred = model(X_train_tensor)
    val_pred = model(X_val_tensor)
    
    # Move tensors to CPU before converting to numpy
    train_pred_original = y_scaler.inverse_transform(train_pred.cpu().numpy())
    val_pred_original = y_scaler.inverse_transform(val_pred.cpu().numpy())
    
    # Calculate mean absolute error in original scale
    train_mae = np.mean(np.abs(train_pred_original - y_train.reshape(-1, 1)))
    val_mae = np.mean(np.abs(val_pred_original - y_val.reshape(-1, 1)))

print(f'\nFinal Results:')
print(f'Training Loss (scaled): {train_losses[-1]:.4f}')
print(f'Validation Loss (scaled): {val_losses[-1]:.4f}')
print(f'Training MAE (original scale): ${train_mae:,.2f}')
print(f'Validation MAE (original scale): ${val_mae:,.2f}')

# Print some example predictions
print("\nSample Predictions vs Actual Values (in original scale):")
for i in range(5):
    print(f"Predicted: ${val_pred_original[i][0]:,.2f}, Actual: ${y_val[i]:,.2f}")


#plot the loss 
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

Using device: cuda
Epoch [10/100], Train Loss: 0.5868, Val Loss: 1.0256
Epoch [20/100], Train Loss: 0.4590, Val Loss: 0.8045
Epoch [30/100], Train Loss: 0.4422, Val Loss: 0.7728
Epoch [40/100], Train Loss: 0.4304, Val Loss: 0.7652
Epoch [50/100], Train Loss: 0.4215, Val Loss: 0.7605
Epoch [60/100], Train Loss: 0.4146, Val Loss: 0.7551
Epoch [70/100], Train Loss: 0.4083, Val Loss: 0.7506
Epoch [80/100], Train Loss: 0.4025, Val Loss: 0.7477
Epoch [90/100], Train Loss: 0.3975, Val Loss: 0.7452
Epoch [100/100], Train Loss: 0.3930, Val Loss: 0.7422


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.